In [5]:
import torch
import torch.nn as nn
import numpy as np
import pickle
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from crystallm import (
    GPT,
    GPTConfig,
    CIFTokenizer
)

/home/uccacbo/miniconda3/envs/crystallm_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
from crystallm import CIFTokenizer  # Assuming CIFTokenizer is defined in crystallm

# Load the tokenizer (assuming a similar API)
tokenizer = CIFTokenizer()

# Assuming the tokens are stored in a dictionary-like attribute
if hasattr(tokenizer, 'token_to_id'):
    num_tokens = len(tokenizer.token_to_id)
    print(f"Number of tokens in the tokenizer: {num_tokens}")




Number of tokens in the tokenizer: 372


In [6]:
# check current directory
import os
os.getcwd()

# change directory to where the data is
os.chdir('/home/uccacbo/CrystaLLM')
os.getcwd()

'/home/uccacbo/CrystaLLM'

In [7]:
!python bin/train.py --config=config/finetune_head_BG.yaml

Using configuration:
out_dir: small_model_untouched
ckpt_out_dir: crystallm_v1_small/BG_fh_2
eval_interval: 250
log_interval: 50
eval_iters_train: 200
eval_iters_val: 200
eval_only: false
always_save_checkpoint: true
init_from: scratch
dataset: BG_cifs_more_tokens
gradient_accumulation_steps: 40
batch_size: 4
block_size: 1024
n_layer: 6
n_head: 6
n_embd: 384
dropout: 0.2
bias: false
learning_rate: 5.0e-05
max_iters: 550
weight_decay: 0.1
beta1: 0.9
beta2: 0.99
grad_clip: 1.0
decay_lr: true
warmup_iters: 10
lr_decay_iters: 550
min_lr: 0.0005
device: cuda
dtype: bfloat16
compile: true
underrep_p: 0.0
validate: false
finetune_head: true

Creating small_model_untouched...
Found vocab_size = 372 (inside BG_cifs_more_tokens/meta.pkl)
Initializing a new model from scratch...
number of parameters: 10.76M
Finetuning head only: Freezing transformer layers
Compiling the model (takes a ~minute)...
[2024-10-18 11:25:28,818] torch._inductor.utils: [WARNING] using triton random, expect difference fro

In [50]:
!python bin/make_prompt_file.py Na2Cl2 prompts/na2cl2.txt --spacegroup P4/nmm

writing prompt to prompts/na2cl2.txt ...


In [5]:
!python bin/sample.py --config prompt_config/BG_gen.yaml

Using configuration:
out_dir: crystallm_v1_small/BG_fh_2
start: FILE:prompts/Fe4H4O8.txt
num_samples: 1
max_new_tokens: 1000
temperature: 0.8
top_k: 10
seed: 1337
device: cuda
dtype: bfloat16
compile: false
target: console
generated_dir: test_generated_cifs_BG

number of parameters: 10.76M
data_Fe4H4O8
loop_
_atom_type_symbol
_atom_type_electronegativity
_atom_type_radius
_atom_type_ionic_radius
Fe 1.8300 1.4000 0.8525
H 2.2000 0.2500 0.0000
O 3.4400 0.6000 1.2600
_symmetry_space_group_name_H-M Pnma
_cell_length_a 14.2326
_cell_length_b 2.9813
_cell_length_c 5.2030
_cell_angle_alpha 90.0000
_cell_angle_beta 90.0000
_cell_angle_gamma 90.0000
_symmetry_Int_Tables_number 62
_chemical_formula_structural FeHO2
_chemical_formula_sum 'Fe4 H4 O8'
_cell_volume 220.7694
_cell_formula_units_Z 4
loop_
_symmetry_equiv_pos_site_id
_symmetry_equiv_pos_as_xyz
1 'x, y, z'
loop_
_atom_site_type_symbol
_atom_site_label
_atom_site_symmetry_multiplicity
_atom_site_fract_x
_atom_site_fract_y
_atom_site_frac

In [45]:
!python bin/postprocess.py test_generated_cifs_BG my_processed_cifs_BG

error post-processing CIF file 'sample_1.cif': could not extract space group from:
data_Co4B2Os2
 loop_ Bandgap_eV: _atom_type_symbol _atom_type_electronegativity _atom_type_radius Co,1z8800,1z3500,0z7683 B,2z0400,0z4500,0z4100 Os,2z2000,1z3000,0z6730 _symmetry_space_group_name_H-M,I4/mcm _cell_length_a,3z6036 _cell_length_b,3z6036 _cell_length_c,6z4358 _cell_angle_alpha,90z0000 _cell_angle_beta,90z0000 _cell_angle_gamma,90z0000 _symmetry_Int_Tables_number,119 _chemical_formula_structural,Co2BOs _chemical_formula_sum,/Co4,B2,Os2/ _cell_volume,83z4773 _cell_formula_units_Z,2 loop_ _symmetry_equiv_pos_site_id _symmetry_equiv_pos_as_xyz 1,/_atom_type_oxidation_number',x',y/ loop_ _atom_site_type_symbol _atom_site_label _atom_site_symmetry_multiplicity _atom_site_fract_x _atom_site_fract_y _atom_site_fract_z _atom_site_occupancy Co,Co0,2,0z0000,0z0000,0z0000,1 Co,Co1,2,0z0000,0z5000,0z7500,1 B,B2,2,0z0000,0z5000,0z2500,1 Os,Os3,2,0z0000,0z0000,0z5000,1  data_Ca4Y4Mg4Al4 loop_ Bandgap_eV: _

### Next steps
1. could be that there is no end to the genertation because somehow in the sample.py file the condition to terminate is not found
    
    a. even the max tokens isnt used

2. Maybe when adding the dimnension I didnt add it correctly so all the dimensions got mumble jumbled so the weights and parameter weights arent assigned to the right thing yet (LoRA could fix this? - but for now could try the token resize thing)

3. Maybe need to retrtain all params from scratch

4. 